In [1]:
#Problema transporte con PulP

In [2]:
#Problema transporte con PulP
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import cvxopt
import pulp
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ
from cvxopt import glpk

In [4]:
# Ejemplo del problema de transporte de las cervezas utilizando PuLP
# Creamos la variable prob que contiene los datos del problema
prob = pulp.LpProblem("Problema de distribución de cerveza", pulp.LpMinimize)

In [5]:
# Creamos lista de cervecerías o nodos de oferta
cervecerias = ["Cervecería A", "Cervercería B"]

# diccionario con la capacidad de oferta de cada cerveceria
oferta = {"Cervecería A": 1000,
          "Cervercería B": 4000}

# Creamos la lista de los bares o nodos de demanda
bares = ["Bar 1", "Bar 2", "Bar 3", "Bar 4", "Bar 5"]

# diccionario con la capacidad de demanda de cada bar
demanda = {"Bar 1":500,
           "Bar 2":900,
           "Bar 3":1800,
           "Bar 4":200,
           "Bar 5":700,}

# Lista con los costos de transporte de cada nodo
costos = [   #Bares
         #1 2 3 4 5
         [2,4,5,2,1],#A   Cervecerías
         [3,1,3,2,3] #B
         ]

# Convertimos los costos en un diccionario de PuLP
costos = pulp.makeDict([cervecerias, bares], costos,0)

# Creamos una lista de tuplas que contiene todas las posibles rutas de tranporte.
rutas = [(c,b) for c in cervecerias for b in bares]

In [6]:
# creamos diccionario x que contendrá la candidad enviada en las rutas
x = pulp.LpVariable.dicts("ruta", (cervecerias, bares), 
                          lowBound = 0,
                          cat = pulp.LpInteger)

# Agregamos la función objetivo al problema
prob += sum([x[c][b]*costos[c][b] for (c,b) in rutas]), \
"Suma_de_costos_de_transporte"

# Agregamos la restricción de máxima oferta de cada cervecería al problema.
for c in cervecerias:
    prob += sum([x[c][b] for b in bares]) <= oferta[c], \
    "Suma_de_Productos_que_salen_de_cervecerias_%s"%c

# Agregamos la restricción de demanda mínima de cada bar
for b in bares:
    prob += sum([x[c][b] for c in cervecerias]) >= demanda[b], \
    "Sum_of_Products_into_Bar%s"%b
                   
# Los datos del problema son exportado a archivo .lp
prob.writeLP("problemaDeTransporte.lp")

# Resolviendo el problema.
prob.solve()

# Imprimimos el estado del problema.
print("Status: {}".format(pulp.LpStatus[prob.status]))

# Imprimimos cada variable con su solución óptima.
for v in prob.variables():
    print("{0:} = {1:}".format(v.name, v.varValue))

# Imprimimos el valor óptimo de la función objetivo   
print("Costo total de transporte = {}".format(prob.objective.value()))

PulpSolverError: PuLP: Error while executing glpsol.exe

In [7]:
#Problema transporte con Pyomo

In [3]:
# Ejemplo del problema de transporte de las cervezas utilizando Pyomo
# Creamos el modelo
modelo = ConcreteModel()

# Creamos los nodos de oferta y demanda
modelo.i = Set(initialize=['Cervecería A',
                           'Cervecería B'],
               doc='Cervecerías')
modelo.j = Set(initialize=['Bar 1', 'Bar 2', 'Bar 3', 'Bar 4', 'Bar 5'],
               doc='Bares')

# Definimos las capacidades de oferta y demanda
modelo.a = Param(modelo.i, initialize={'Cervecería A':1000,'Cervecería B':4000},
                 doc='Capacidad de oferta de las cervecerías')
modelo.b = Param(modelo.j, initialize={'Bar 1':500,'Bar 2':900,'Bar 3':1800, 
                                      'Bar 4':200, 'Bar 5':700 },
                 doc='Demanda de cada bar')

# Costo de transporte
costos = {
    ('Cervecería A', 'Bar 1'): 2,
    ('Cervecería A', 'Bar 2'): 4,
    ('Cervecería A', 'Bar 3'): 5,
    ('Cervecería A', 'Bar 4'): 2,
    ('Cervecería A', 'Bar 5'): 1,
    ('Cervecería B', 'Bar 1'): 3,
    ('Cervecería B', 'Bar 2'): 1,
    ('Cervecería B', 'Bar 3'): 3,
    ('Cervecería B', 'Bar 4'): 2,
    ('Cervecería B', 'Bar 5'): 3
    }
modelo.d = Param(modelo.i, modelo.j, initialize=costos, 
                doc='Costo de transporte')

# definimos el costo de tranporte
def f_costo(modelo, i, j):
    return modelo.d[i,j]
modelo.c = Param(modelo.i, modelo.j, initialize=f_costo, 
                doc='Costo de transporte')

# definimos variable x con las cantidades de cajas enviadas
modelo.x = Var(modelo.i, modelo.j, bounds=(0.0,None), 
              doc='Cantidad de cajas')

In [4]:
## Definimos las restricciones ##
# Límite de oferta
def f_oferta(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.j) <= modelo.a[i]
modelo.oferta = Constraint(modelo.i, rule=f_oferta, 
                           doc='Límites oferta de cada Cervecería')

# Límite de demanda
def f_demanda(modelo, j):
    return sum(modelo.x[i,j] for i in modelo.i) >= modelo.b[j]  
modelo.demanda = Constraint(modelo.j, rule=f_demanda, 
                            doc='Límites demanda de cada bar')

In [5]:
## Definimos la función objetivo y resolvemos el problema ##
# Función objetivo
def f_objetivo(modelo):
    return sum(modelo.c[i,j]*modelo.x[i,j] for i in modelo.i for j in modelo.j)
modelo.objetivo = Objective(rule=f_objetivo, sense=minimize, 
                           doc='Función Objetivo')

# resolvemos el problema e imprimimos resultados
def pyomo_postprocess(options=None, instance=None, results=None):
    modelo.x.display()

# utilizamos solver glpk
opt = SolverFactory("glpk")
resultados = opt.solve(modelo)

# imprimimos resultados
print("\nSolución óptima encontrada\n" + '-'*80)
pyomo_postprocess(None, None, resultados)


Solución óptima encontrada
--------------------------------------------------------------------------------
x : Cantidad de cajas
    Size=10, Index=x_index
    Key                       : Lower : Value  : Upper : Fixed : Stale : Domain
    ('Cervecería A', 'Bar 1') :   0.0 :  300.0 :  None : False : False :  Reals
    ('Cervecería A', 'Bar 2') :   0.0 :    0.0 :  None : False : False :  Reals
    ('Cervecería A', 'Bar 3') :   0.0 :    0.0 :  None : False : False :  Reals
    ('Cervecería A', 'Bar 4') :   0.0 :    0.0 :  None : False : False :  Reals
    ('Cervecería A', 'Bar 5') :   0.0 :  700.0 :  None : False : False :  Reals
    ('Cervecería B', 'Bar 1') :   0.0 :  200.0 :  None : False : False :  Reals
    ('Cervecería B', 'Bar 2') :   0.0 :  900.0 :  None : False : False :  Reals
    ('Cervecería B', 'Bar 3') :   0.0 : 1800.0 :  None : False : False :  Reals
    ('Cervecería B', 'Bar 4') :   0.0 :  200.0 :  None : False : False :  Reals
    ('Cervecería B', 'Bar 5') :   0.0 :   

In [6]:
#Problema transporte con Pyomo - Modelo de transporte estandar - equilibrado

In [7]:
# Ejemplo del problema de transporte de las Plantas de produccion utilizando Pyomo
# Creamos el modelo
modelo = ConcreteModel()

# Creamos los nodos de oferta y demanda
modelo.i = Set(initialize=['Los Angeles','Detroit','Nueva Orleáns'],doc='Origenes')
modelo.j = Set(initialize=['Denver','Miami'],doc='Destinos')

# Definimos las capacidades de oferta y demanda
modelo.a = Param(modelo.i, initialize={'Los Angeles':1000,'Detroit':1500,'Nueva Orleáns':1200},
                 doc='Centros de produccion')
modelo.b = Param(modelo.j, initialize={'Denver':2300,'Miami':1400},
                 doc='Demanda de cada bar')

# Costo de transporte
costos = {
    ('Los Angeles', 'Denver'): 80,
    ('Los Angeles', 'Miami'): 215,
    ('Detroit', 'Denver'): 100,
    ('Detroit', 'Miami'): 108,
    ('Nueva Orleáns', 'Denver'): 102,
    ('Nueva Orleáns', 'Miami'): 68
     }
modelo.d = Param(modelo.i, modelo.j, initialize=costos,  doc='Costo de transporte')

# definimos el costo de tranporte
def f_costo(modelo, i, j):
    return modelo.d[i,j]
modelo.c = Param(modelo.i, modelo.j, initialize=f_costo, doc='Costo de transporte')

# definimos variable x con las cantidades de cajas enviadas
modelo.x = Var(modelo.i, modelo.j, bounds=(0.0,None), doc='Cantidad de cajas')

In [8]:
## Definimos las restricciones ##
# Límite de oferta
def f_oferta(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.j) <= modelo.a[i]
modelo.oferta = Constraint(modelo.i, rule=f_oferta, 
                           doc='Límites oferta de cada Planta')

# Límite de demanda
def f_demanda(modelo, j):
    return sum(modelo.x[i,j] for i in modelo.i) >= modelo.b[j]  
modelo.demanda = Constraint(modelo.j, rule=f_demanda, 
                            doc='Límites demanda de cada bar')

In [9]:
## Definimos la función objetivo y resolvemos el problema ##
# Función objetivo
def f_objetivo(modelo):
    return sum(modelo.c[i,j]*modelo.x[i,j] for i in modelo.i for j in modelo.j)
modelo.objetivo = Objective(rule=f_objetivo, sense=minimize, 
                           doc='Función Objetivo')

# resolvemos el problema e imprimimos resultados
def pyomo_postprocess(options=None, instance=None, results=None):
    modelo.x.display()

# utilizamos solver glpk
opt = SolverFactory("glpk")
resultados = opt.solve(modelo)

# imprimimos resultados
print("\nSolución óptima encontrada\n" + '-'*80)
pyomo_postprocess(None, None, resultados)


Solución óptima encontrada
--------------------------------------------------------------------------------
x : Cantidad de cajas
    Size=6, Index=x_index
    Key                         : Lower : Value  : Upper : Fixed : Stale : Domain
          ('Detroit', 'Denver') :   0.0 : 1300.0 :  None : False : False :  Reals
           ('Detroit', 'Miami') :   0.0 :  200.0 :  None : False : False :  Reals
      ('Los Angeles', 'Denver') :   0.0 : 1000.0 :  None : False : False :  Reals
       ('Los Angeles', 'Miami') :   0.0 :    0.0 :  None : False : False :  Reals
    ('Nueva Orleáns', 'Denver') :   0.0 :    0.0 :  None : False : False :  Reals
     ('Nueva Orleáns', 'Miami') :   0.0 : 1200.0 :  None : False : False :  Reals


In [50]:
#Problema transporte con PulP - Problema del transporte desequilibrado en oferta_artículos

In [ ]:
#Minimize pulp.LpMinimize

In [ ]:
# IMPORTANDO MÓDULOS NECESARIOS y TÍPICOS IO
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import cvxopt
import pulp
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ

In [ ]:
##Ejemplo del problema de transporte de artículos desequilibrado utilizando PuLP
# Creamos la variable prob que contiene los datos del problema
prob = pulp.LpProblem("Problema de transporte de artículos a mayoristas", pulp.LpMinimize)

In [7]:
# Creamos lista de Plantas de producción o nodos de oferta
Plantas = ["A1", "A2", "A3", "Ficticia"]
# diccionario con la capacidad de oferta de cada Planta de producción
oferta = {"A1": 1500,
          "A2": 1500,
          "A3": 1500,
          "Ficticia": 200}
# Creamos la lista de los Centros de distribución o nodos de demanda
Centros = ["C1", "C2", "C3", "C4"]
# diccionario con la capacidad de demanda de cada Centro de distribución
demanda = {"C1": 1000,
           "C2": 1200,
           "C3": 1500,
           "C4": 1000}
# Lista con los costos de transporte de cada nodo
costos = [ #Centros de distribución
    # C1 C2 C3 C4
    [30, 10, 25, 20],# Planta A1
    [15, 25, 30, 10], # Planta A2
    [20, 30, 15, 20], # Planta A3
    [0, 0, 0, 0] # Planta Ficticia
]

In [8]:
# Convertimos los costos en un diccionario de PuLP
costos = pulp.makeDict([Plantas, Centros], costos,0)
# Creamos una lista de tuplas que contiene todas las posibles rutas de tranporte.
rutas = [(c,b) for c in Plantas for b in Centros]

In [10]:
# creamos diccionario x que contendrá la candidad enviada en las rutas
x = pulp.LpVariable.dicts("ruta", (Plantas, Centros),
                          lowBound = 0,
                          cat = pulp.LpInteger)
# Agregamos la función objetivo al problema
prob += sum([x[c][b]*costos[c][b] for (c,b) in rutas]), \
"Suma_de_costos_de_transporte"
# Agregamos la restricción de máxima oferta de cada Planta al problema.
for c in Plantas:
    prob += sum([x[c][b] for b in Centros]) <= oferta[c], \
    "Suma_de_Productos_que_salen_de_Plantas_%s"%c
# Agregamos la restricción de demanda mínima de cada Centro
for b in Centros:
    prob += sum([x[c][b] for c in Plantas]) >= demanda[b], \
    "Sum_of_Products_into_Bar%s"%b
# Los datos del problema son exportado a archivo .lp
prob.writeLP("problemaDeTransporte.lp")
# Resolviendo el problema.
prob.solve()
# Imprimimos el estado del problema.
print("Status: {}".format(pulp.LpStatus[prob.status]))
# Imprimimos cada variable con su solución óptima.
for v in prob.variables():
    print("{0:} = {1:}".format(v.name, v.varValue))
# Imprimimos el valor óptimo de la función objetivo
print("Coste total de transporte = {}".format(prob.objective.value()))

PulpError: overlapping constraint names: Suma_de_Productos_que_salen_de_Plantas_A1

In [18]:
#Ejemplo 2: Problema transporte con Pyomo - Modelo de transporte con oferta menor que la demandad esequilibrio)

In [29]:
##Ejemplo del problema de transporte de artículos desequilibrado utilizando PuLP
# Creamos la variable prob que contiene los datos del problema
prob = pulp.LpProblem("Problema de transporte de artículos a mayoristas", pulp.LpMinimize)

In [35]:
# Ejemplo del problema de transporte de las Plantas de produccion utilizando Pyomo
# Creamos el modelo
modelo = ConcreteModel()

# Creamos los nodos de oferta y demanda
modelo.i = Set(initialize=['Los Angeles','Detroit','Nueva Orleáns','Ficticia'],doc='Origenes')
modelo.j = Set(initialize=['Denver','Miami'],doc='Destinos')

# Definimos las capacidades de oferta y demanda
modelo.a = Param(modelo.i, initialize={'Los Angeles':1000,'Detroit':1300,'Nueva Orleáns':1200,'Ficticia':200},
                 doc='Centros de produccion')
modelo.b = Param(modelo.j, initialize={'Denver':2300,'Miami':1400},
                 doc='Centros de distribucion')

# Costo de transporte
costos = {
    ('Los Angeles', 'Denver'): 80,
    ('Los Angeles', 'Miami'): 215,
    ('Detroit', 'Denver'): 100,
    ('Detroit', 'Miami'): 108,
    ('Nueva Orleáns', 'Denver'): 102,
    ('Nueva Orleáns', 'Miami'): 68,
    ('Ficticia','Denver'):0,
    ('Ficticia','Miami'):0}
modelo.d = Param(modelo.i, modelo.j, initialize=costos,  doc='Costo de transporte')

# definimos el costo de tranporte
def f_costo(modelo, i, j):
    return modelo.d[i,j]
modelo.c = Param(modelo.i, modelo.j, initialize=f_costo, doc='Costo de transporte')

# definimos variable x con las cantidades de cajas enviadas
modelo.x = Var(modelo.i, modelo.j, bounds=(0.0,None), doc='Cantidad de cajas')

In [36]:
## Definimos las restricciones ##
# Límite de oferta
def f_oferta(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.j) <= modelo.a[i]
modelo.oferta = Constraint(modelo.i, rule=f_oferta, 
                           doc='Límites oferta dde cada Fabrica')

# Límite de demanda
def f_demanda(modelo, j):
    return sum(modelo.x[i,j] for i in modelo.i) >= modelo.b[j]  
modelo.demanda = Constraint(modelo.j, rule=f_demanda, 
                            doc='Límites demanda de cada Concesionario')

In [37]:
## Definimos la función objetivo y resolvemos el problema ##
# Función objetivo
def f_objetivo(modelo):
    return sum(modelo.c[i,j]*modelo.x[i,j] for i in modelo.i for j in modelo.j)
modelo.objetivo = Objective(rule=f_objetivo, sense=minimize, 
                           doc='Función Objetivo')

# resolvemos el problema e imprimimos resultados
def pyomo_postprocess(options=None, instance=None, results=None):
    modelo.x.display()

# utilizamos solver glpk
opt = SolverFactory("glpk")
resultados = opt.solve(modelo)

# imprimimos resultados
print("\nSolución óptima encontrada\n" + '-'*80)
pyomo_postprocess(None, None, resultados)


Solución óptima encontrada
--------------------------------------------------------------------------------
x : Cantidad de cajas
    Size=8, Index=x_index
    Key                         : Lower : Value  : Upper : Fixed : Stale : Domain
          ('Detroit', 'Denver') :   0.0 : 1300.0 :  None : False : False :  Reals
           ('Detroit', 'Miami') :   0.0 :    0.0 :  None : False : False :  Reals
         ('Ficticia', 'Denver') :   0.0 :    0.0 :  None : False : False :  Reals
          ('Ficticia', 'Miami') :   0.0 :  200.0 :  None : False : False :  Reals
      ('Los Angeles', 'Denver') :   0.0 : 1000.0 :  None : False : False :  Reals
       ('Los Angeles', 'Miami') :   0.0 :    0.0 :  None : False : False :  Reals
    ('Nueva Orleáns', 'Denver') :   0.0 :    0.0 :  None : False : False :  Reals
     ('Nueva Orleáns', 'Miami') :   0.0 : 1200.0 :  None : False : False :  Reals


In [44]:
#Problema transporte con Pyomo - Modelo de transporte con oferta mayor que la demanda-desequilibrio

In [49]:
# Ejemplo del problema de transporte de las Plantas de produccion utilizando Pyomo
# Creamos el modelo
modelo = ConcreteModel()

# Creamos los nodos de oferta y demanda
modelo.i = Set(initialize=['Los Angeles','Detroit','Nueva Orleáns'],doc='Origenes')
modelo.j = Set(initialize=['Denver','Miami'],doc='Destinos')

# Definimos las capacidades de oferta y demanda
modelo.a = Param(modelo.i, initialize={'Los Angeles':1000,'Detroit':1500,'Nueva Orleáns':1200},
                 doc='Centros de produccion')
modelo.b = Param(modelo.j, initialize={'Denver':1900,'Miami':1400},
                 doc='Centros de distribucion')

# Costo de transporte
costos = {
    ('Los Angeles', 'Denver'): 80,
    ('Los Angeles', 'Miami'): 215,
    ('Detroit', 'Denver'): 100,
    ('Detroit', 'Miami'): 108,
    ('Nueva Orleáns', 'Denver'): 102,
    ('Nueva Orleáns', 'Miami'): 68}
modelo.d = Param(modelo.i, modelo.j, initialize=costos,  doc='Costo de transporte')

# definimos el costo de tranporte
def f_costo(modelo, i, j):
    return modelo.d[i,j]
modelo.c = Param(modelo.i, modelo.j, initialize=f_costo, doc='Costo de transporte')

# definimos variable x con las cantidades de cajas enviadas
modelo.x = Var(modelo.i, modelo.j, bounds=(0.0,None), doc='Cantidad de cajas')

In [50]:
## Definimos las restricciones ##
# Límite de oferta
def f_oferta(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.j) <= modelo.a[i]
modelo.oferta = Constraint(modelo.i, rule=f_oferta, 
                           doc='Límites oferta de cada Cervecería')

# Límite de demanda
def f_demanda(modelo, j):
    return sum(modelo.x[i,j] for i in modelo.i) >= modelo.b[j]  
modelo.demanda = Constraint(modelo.j, rule=f_demanda, 
                            doc='Límites demanda de cada bar')

In [51]:
## Definimos la función objetivo y resolvemos el problema ##
# Función objetivo
def f_objetivo(modelo):
    return sum(modelo.c[i,j]*modelo.x[i,j] for i in modelo.i for j in modelo.j)
modelo.objetivo = Objective(rule=f_objetivo, sense=minimize, 
                           doc='Función Objetivo')

# resolvemos el problema e imprimimos resultados
def pyomo_postprocess(options=None, instance=None, results=None):
    modelo.x.display()

# utilizamos solver glpk
opt = SolverFactory("glpk")
resultados = opt.solve(modelo)

# imprimimos resultados
print("\nSolución óptima encontrada\n" + '-'*80)
pyomo_postprocess(None, None, resultados)


Solución óptima encontrada
--------------------------------------------------------------------------------
x : Cantidad de cajas
    Size=6, Index=x_index
    Key                         : Lower : Value  : Upper : Fixed : Stale : Domain
          ('Detroit', 'Denver') :   0.0 :  900.0 :  None : False : False :  Reals
           ('Detroit', 'Miami') :   0.0 :  200.0 :  None : False : False :  Reals
      ('Los Angeles', 'Denver') :   0.0 : 1000.0 :  None : False : False :  Reals
       ('Los Angeles', 'Miami') :   0.0 :    0.0 :  None : False : False :  Reals
    ('Nueva Orleáns', 'Denver') :   0.0 :    0.0 :  None : False : False :  Reals
     ('Nueva Orleáns', 'Miami') :   0.0 : 1200.0 :  None : False : False :  Reals
